## CH-04: Beginning Data Analysis 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 6, 'display.max_rows', 6, 'display.max_colwidth', 12)

## Introduction

## Developing a data analysis routine

In [ ]:
college = pd.read_csv('../data/college.csv')
college.sample(random_state=42)
college.head()

In [ ]:
# Get the dimensions of the DataFrame with the .shape attribute:

college.shape

In [ ]:
# List the data type of each column, the number of non-missing values, and memory
# usage with the .info method:

college.info()

In [ ]:
college.describe()

In [ ]:
# Get summary statistics for the numerical columns and transpose the DataFrame 
# for more readable output:

college.describe(include=[np.number]).T

In [ ]:
college.describe(include=[pd.Categorical]).T

In [ ]:
college.describe(include=[object]).T

In [ ]:
# It is possible to specify the exact quantiles returned from the .describe method 
# when used with numeric columns:

college.describe(include=[np.number],
   percentiles=[.01, .05, .10, .25, .5,
                .75, .9, .95, .99]).T

## Data dictionaries

In [ ]:
# A crucial part of data analysis involves creating and maintaining a data dictionary. 

# A data dictionary is a table of metadata and notes on each column of data. One of the 
# primary purposes of a data dictionary is to explain the meaning of the column names. 

# The college dataset uses a lot of abbreviations that are likely to be unfamiliar to an 
# analyst who is inspecting it for the first time.


In [ ]:
# A data dictionary for the college dataset is provided in the following college_data_
# dictionary.csv file:

pd.read_csv('../data/college_data_dictionary.csv')

## Reducing memory by changing data types

In [ ]:
# After reading in our college dataset, we select a few columns of different data types
# that will clearly show how much memory may be saved:

In [ ]:
college = pd.read_csv('../data/college.csv')
different_cols = ['RELAFFIL', 'SATMTMID', 'CURROPER', 'INSTNM', 'STABBR']

In [ ]:
# This command is selecting all rows from the DataFrame college but only 
# the columns specified in the different_cols variable. 

# The variable different_cols is expected to contain the column labels that 
# you want to extract from the DataFrame.

col2 = college.loc[:, different_cols]
col2.head()

In [ ]:
#  Inspect the data types of each column:

col2.dtypes

In [ ]:
# Find the memory usage of each column with the .memory_usage method:

original_mem = col2.memory_usage(deep=True)
original_mem

In [ ]:
# There is no need to use 64 bits for the RELAFFIL column as it contains only 0 or 1.
# Let's convert this column to an 8-bit (1 byte) integer with the .astype method:

col2['RELAFFIL'] = col2['RELAFFIL'].astype(np.int8)    

In [ ]:
col2.dtypes

In [ ]:
college[different_cols].memory_usage(deep=True)

In [ ]:
# To save even more memory, you will want to consider changing object data types to
# categorical if they have a reasonably low cardinality (number of unique values). Let's
# first check the number of unique values for both the object columns:


col2.select_dtypes(include=['object']).nunique()

In [ ]:
# The STABBR column is a good candidate to convert to categorical as less than one
# percent of its values are unique:

col2['STABBR'] = col2['STABBR'].astype('category')
col2.dtypes

In [ ]:
# Compute the memory usage again:

new_mem = col2.memory_usage(deep=True)
new_mem

In [ ]:
# Finally, let's compare the original memory usage with our updated memory usage.
# The RELAFFIL column is, as expected, an eighth of its original size, while the
# STABBR column has shrunk to just three percent of its original size:

new_mem / original_mem

## Selecting the smallest of the largest

In [2]:
# This recipe can be used to create catchy news headlines such as Out of the Top 100
# Universities, These 5 have the Lowest Tuition, or From the Top 50 Cities to Live, 
# these 10 are the Most Affordable.

In [3]:
# During analysis, it is possible that you will first need to find a grouping of data 
# that contains the top n values in a single column and, from this subset, find the 
# bottom m values based on a different column.

In [ ]:
# In this recipe, we find the five lowest budget movies from the top 100 scoring 
# movies by taking advantage of the convenience methods: .nlargest and .nsmallest.

In [4]:
movie = pd.read_csv('../data/movie.csv')
movie2 = movie[['movie_title', 'imdb_score', 'budget']]
movie2.head()

,movie_title,imdb_score,budget
0,Avatar,7.9,237000000.0
1,Pirates ...,7.1,300000000.0
2,Spectre,6.8,245000000.0
3,The Dark...,8.5,250000000.0
4,Star War...,7.1,NaN


In [5]:
# Use the .nlargest method to select the top 100 movies by imdb_score

movie2.nlargest(100, 'imdb_score').head()

,movie_title,imdb_score,budget
2725,Towering...,9.5,NaN
1920,The Shaw...,9.3,25000000.0
3402,The Godf...,9.2,6000000.0
2779,Dekalog,9.1,NaN
4312,Kickboxe...,9.1,17000000.0


In [6]:
# Chain the .nsmallest method to return the five lowest budget films among those with a top 100 score:

(movie2
  .nlargest(100, 'imdb_score')
  .nsmallest(5, 'budget')
)

,movie_title,imdb_score,budget
4804,Butterfl...,8.7,180000.0
4801,Children...,8.5,180000.0
4706,12 Angry...,8.9,350000.0
4550,A Separa...,8.4,500000.0
4636,The Othe...,8.4,500000.0


In [ ]:
# The first parameter of the .nlargest method, n, must be an integer and selects the 
# number of rows to be returned. The second parameter, columns, takes a column name 
# as a string.

# Step 2 returns the 100 highest-scoring movies. We could have saved this intermediate 
# result as its own variable but instead, we chain the .nsmallest method to it in step 3, 
# which returns exactly five rows, sorted by budget.

In [ ]:
# It is possible to pass a list of column names to the columns parameter of the .nlargest
# and .nsmallest methods. This would only be useful to break ties in the event that there
# were duplicate values sharing the nth ranked spot in the first column in the list.

## Selecting the largest of each group by sorting

In [ ]:
# One of the most basic and common operations to perform during data analysis is to select
# rows containing the largest value of some column within a group. For instance, this would be
# like finding the highest-rated film of each year or the highest-grossing film by content rating.

# To accomplish this task, we need to sort the groups as well as the column used to rank each
# member of the group, and then extract the highest member of each group.

# In this recipe, we will find the highest-rated film of each year.

In [ ]:
movie = pd.read_csv('../data/movie.csv')
movie[['movie_title', 'title_year', 'imdb_score']]

In [ ]:
# Use the .sort_values method to sort the DataFrame by title_year. The default behavior sorts from the smallest to the largest. 
# Use the ascending=True parameter to invert this behavior:

(movie
  [['movie_title', 'title_year', 'imdb_score']]
  .sort_values('title_year', ascending=False)
)

In [ ]:
# To sort multiple columns at once, use a list.

(movie
  [['movie_title', 'title_year', 'imdb_score']]
  .sort_values(['title_year','imdb_score'],
               ascending=False)
)

In [ ]:
# We use the .drop_duplicates method to keep only the first row of every year:

(movie
  [['movie_title', 'title_year', 'imdb_score']]
  .sort_values(['title_year','imdb_score'],
               ascending=False)
  .drop_duplicates(subset='title_year')
)

### How it works... Group by Operations

##  There's more...

In [ ]:
(movie
  [['movie_title', 'title_year', 'imdb_score']]
  .groupby('title_year', as_index=False)
  .apply(lambda df: df.sort_values('imdb_score',
         ascending=False).head(1))
  .sort_values('title_year', ascending=False)
)

In [ ]:
(movie
  [['movie_title', 'title_year',
    'content_rating', 'budget']]
   .sort_values(['title_year',
       'content_rating', 'budget'],
       ascending=[False, False, True])
   .drop_duplicates(subset=['title_year',
        'content_rating'])
)

## Replicating nlargest with sort_values

### How to do it...

In [ ]:
# Use .sort_values to replicate the first part of the expression and grab the first 100 rows with the .head method

movie = pd.read_csv('../data/movie.csv')
(movie
   [['movie_title', 'imdb_score', 'budget']]
   .nlargest(100, 'imdb_score') 
   .nsmallest(5, 'budget')
)

In [ ]:
(movie
   [['movie_title', 'imdb_score', 'budget']]
   .sort_values('imdb_score', ascending=False)
   .head(100)
)

In [ ]:
# Now that we have the top 100 scoring movies, we can use .sort_values with .head again to grab the lowest five by budget

(movie
   [['movie_title', 'imdb_score', 'budget']]
   .sort_values('imdb_score', ascending=False)
   .head(100) 
   .sort_values('budget')
   .head(5)
)

### How it works...

In [ ]:
(movie
   [['movie_title', 'imdb_score', 'budget']]
   .nlargest(100, 'imdb_score')
   .tail()
)

In [ ]:
(movie
   [['movie_title', 'imdb_score', 'budget']]
   .sort_values('imdb_score', ascending=False) 
   .head(100)
   .tail()
)

## Calculating a trailing stop order price

### How to do it...

In [ ]:
import datetime
#import pandas_datareader.data as web
import requests_cache
session = requests_cache.CachedSession(
   cache_name='cache', backend='sqlite', 
   expire_after=datetime.timedelta(days=90))

In [ ]:
tsla = web.DataReader('tsla', data_source='yahoo',
   start='2017-1-1', session=session)
tsla.head(8)

In [ ]:
tsla_close = tsla['Close']

In [ ]:
tsla_cummax = tsla_close.cummax()
tsla_cummax.head()

In [ ]:
(tsla
  ['Close']
  .cummax()
  .mul(.9)
  .head()
)

### How it works...

### There's more...